# An Introduction to Football Player Dashboards Using Python and Matplotlib

From the blog [post](https://medium.com/geekculture/an-introduction-to-football-player-dashboards-using-python-and-matplotlib-eb5328dfe85f) at Medium

Link to original jupyter notebook: [GitHub](https://github.com/shreyas7kha/DataVizTutorial/blob/main/DataVizProj.ipynb)

In [74]:
# Imports
import requests
from bs4 import BeautifulSoup as soup
from bs4 import Comment
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl

### Scraping from Understat

In [2]:
# ALEX EROKHIN'S PLAYER ID IN UNDERSTAT IS 2772
url = 'https://understat.com/player/2772'
html = requests.get(url)
parse_soup = soup(html.content, 'lxml')

In [3]:
scripts = parse_soup.find_all('script')
shots_data_script = scripts[3].string

ind_start = shots_data_script.index("('")+2
ind_end = shots_data_script.index("')")

shots_data_json = shots_data_script[ind_start:ind_end]
shots_data_json = shots_data_json.encode('utf8').decode('unicode_escape')

In [4]:
data = json.loads(shots_data_json)

In [5]:
x = []
y = []
xg = []
result = []
season = []

In [6]:
for i in range(len(data)):
    for key in data[i]:
        if key=='X':
            x.append(data[i][key])
        if key=='Y':
            y.append(data[i][key])
        if key=='xG':
            xg.append(data[i][key])
        if key=='result':
            result.append(data[i][key])
        if key=='season':
            season.append(data[i][key])

columns = ['X','Y','xG','Result','Season']
df_understat = pd.DataFrame([x, y, xg, result, season], index=columns)
df_understat = df_understat.T
df_understat = df_understat.apply(pd.to_numeric,errors='ignore')

In [7]:
df_understat

,X,Y,xG,Result,Season
0,0.759,0.643,0.016020,SavedShot,2014
1,0.740,0.707,0.009415,MissedShots,2014
2,0.814,0.414,0.052361,MissedShots,2014
3,0.747,0.475,0.021899,MissedShots,2014
4,0.817,0.363,0.058956,BlockedShot,2014
...,...,...,...,...,...
315,0.986,0.540,0.789327,MissedShots,2021
316,0.816,0.557,0.050389,Goal,2021
317,0.940,0.353,0.103931,Goal,2021
318,0.836,0.503,0.082823,MissedShots,2021


In [8]:
df_understat['X'] = df_understat['X'].apply(lambda x:x*100)
df_understat['Y'] = df_understat['Y'].apply(lambda x:x*100)

In [9]:
df_understat

,X,Y,xG,Result,Season
0,75.900002,64.300003,0.016020,SavedShot,2014
1,74.000000,70.699997,0.009415,MissedShots,2014
2,81.400002,41.400002,0.052361,MissedShots,2014
3,74.699997,47.500000,0.021899,MissedShots,2014
4,81.699997,36.299999,0.058956,BlockedShot,2014
...,...,...,...,...,...
315,98.599998,54.000000,0.789327,MissedShots,2021
316,81.599998,55.700001,0.050389,Goal,2021
317,94.000000,35.299999,0.103931,Goal,2021
318,83.599998,50.299999,0.082823,MissedShots,2021


### Scraping from Fbref

In [83]:
fbref_html = requests.get('https://fbref.com/en/comps/30/shooting/Russian-Premier-League-Stats')
fbref_soup = soup(fbref_html.content, 'lxml')

In [87]:
comments = fbref_soup.find_all(text=lambda text:isinstance(text, Comment))
# Таблица со статистикой ударов изначально закомментирована по какой-то причине
df_fbref = pd.read_html(comments[23])

In [90]:
df_fbref[0]

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
                    Rk             Player             Nation   
0                    1  Oliver Abildgaard             dk DEN   
1                    2       Arsen Adamov             ru RUS   
2                    3     Gamid Agalarov             ru RUS   
3                    4      Mikhail Ageev             ru RUS   
4                    5     Ilzat Akhmetov             ru RUS   
..                 ...                ...                ...   
305                295    Yuri Zhuravlyov             ru RUS   
306                296   Anton Zinkovskiy             ru RUS   
307                297       Roman Zobnin             ru RUS   
308                298       Georgi Zotov             ru RUS   
309                299    Aleksandr Zuyev             ru RUS   

    Unnamed: 3_level_0 Unnamed: 4_level_0 Unnamed: 5_level_0  \
                   Pos              Squad                Age   
0                   MF        Rubin Kazan             25-060   
1                   DF               Ural             21-293   
2                   FW                Ufa             21-024   
3                   FW               Ural             21-109   
4                MF,FW        CSKA Moscow             23-221   
..                 ...                ...                ...   
305                 DF                Ufa             25-041   
306                 MF             Samara             25-117   
307                 MF     Spartak Moscow             27-179   
308                 DF        Rubin Kazan             31-209   
309                 DF        Rubin Kazan             25-044   

    Unnamed: 6_level_0 Unnamed: 7_level_0 Unnamed: 8_level_0 Standard      \
                  Born                90s                Gls       Sh SoT   
0                 1996                2.0                  0        4   1   
1                 1999                2.0                  0        0   0   
2                 2000                1.4                  2        3   2   
3                 2000                0.7                  0        4   1   
4                 1997                2.4                  0        5   0   
..                 ...                ...                ...      ...  ..   
305               1996                3.0                  0        3   1   
306               1996                2.8                  0        9   0   
307               1994                2.2                  0        4   2   
308               1990                0.6                  0        0   0   
309               1996                1.5                  0        0   0   

                                        Performance       Unnamed: 19_level_0  
     SoT% Sh/90 SoT/90  G/Sh G/SoT Dist          PK PKatt             Matches  
0    25.0  2.00   0.50  0.00  0.00  NaN           0     0             Matches  
1     NaN  0.00   0.00   NaN   NaN  NaN           0     0             Matches  
2    66.7  2.14   1.43  0.67  1.00  NaN           0     0             Matches  
3    25.0  6.10   1.53  0.00  0.00  NaN           0     0             Matches  
4     0.0  2.12   0.00  0.00   NaN  NaN           0     0             Matches  
..    ...   ...    ...   ...   ...  ...         ...   ...                 ...  
305  33.3  1.00   0.33  0.00  0.00  NaN           0     0             Matches  
306   0.0  3.18   0.00  0.00   NaN  NaN           0     0             Matches  
307  50.0  1.81   0.90  0.00  0.00  NaN           0     0             Matches  
308   NaN  0.00   0.00   NaN   NaN  NaN           0     0             Matches  
309   NaN  0.00   0.00   NaN   NaN  NaN           0     0             Matches  

[310 rows x 20 columns]